In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import scipy as sp
import string
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

import nltk
import scikitplot as skplt
import re, string
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
STOPWORDS = set(stopwords.words('russian'))
#list of special characters.You can use regular expressions too

# importing the PorterStemmer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize,word_tokenize
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
#importing the CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import WordPunctTokenizer
from nltk.stem import SnowballStemmer
from pymorphy2 import MorphAnalyzer

# start text processing with vectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\150ho\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\150ho\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\150ho\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [6]:
# !pip3 install pymorphy2

In [9]:
# !pip3 install scikit-plot

In [2]:
ps = SnowballStemmer(language="russian")
lemmatizer = MorphAnalyzer()

In [2]:
data_train = pd.read_json('train.json')
data_train

,text,id,sentiment
0,Досудебное расследование по факту покупки ЕНПФ...,1945,negative
1,Медики рассказали о состоянии пострадавшего му...,1957,negative
2,"Прошел почти год, как железнодорожным оператор...",1969,negative
3,По итогам 12 месяцев 2016 года на территории р...,1973,negative
4,Астана. 21 ноября. Kazakhstan Today - Агентств...,1975,negative
...,...,...,...
8258,"Как мы писали еще весной, для увеличения сбыта...",10312,positive
8259,Но молодой министр национальной экономики Биши...,10313,negative
8260,\n \nВ ЕНПФ назначен новый председатель правле...,10314,neutral
8261,В Алматы у отделения банка произошло нападение...,10315,negative


In [3]:
data_test = pd.read_json('test.json')
data_test

,text,id
0,"Как сообщает пресс-служба акимата Алматы, для ...",0
1,Казахстанские авиакомпании перевозят 250 тысяч...,1
2,На состоявшемся под председательством Касым-Жо...,2
3,В ОАЭ состоялись переговоры между казахстанско...,3
4,12 вагонов грузового поезда сошли с путей в Во...,4
...,...,...
2051,На официальной странице общественного движения...,2079
2052,"официальный курс – 330,55 тенге за Доллар США ...",2083
2053,"«Базовая ставка, которая сейчас составляет 12%...",2084
2054,На начальном этапе за неоплату парковки на при...,2087


In [4]:
%%time
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]+|[\d]+', r'',text).strip()
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    text = regex.sub(' ', text)
#     text = re.sub(r'[^0-9a-zA-Z]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = " ".join(word for word in text.split() if word not in STOPWORDS)
    text = " ".join(ps.stem(lemmatizer.normal_forms(word)[0]) for word in text.split())
    return text
  
data_train['clean_text'] = data_train['text'].apply(clean_text)
data_train.head()

In [6]:
#define a function to get rid of stopwords present in the messages
def message_text_process(mess):
    # Check characters to see if there are punctuations 
    no_punctuation=[char for char in mess if char not in string.punctuation]
    # now form the sentence
    no_punctuation=''.join(no_punctuation)
    # Now eliminate any stopwords
    return[word for word in no_punctuation.split() if word.lower() not in stopwords.words('english') and stopwords.words('russian')]

# to verify that function is working
# data_train['text'].head().apply(message_text_process)

In [7]:
%%time
data = data_train['text'].apply(message_text_process)
data

0       [Досудебное, расследование, по, факту, покупки...
1       [Медики, рассказали, о, состоянии, пострадавше...
2       [Прошел, почти, год, как, железнодорожным, опе...
3       [По, итогам, 12, месяцев, 2016, года, на, терр...
4       [Астана, 21, ноября, Kazakhstan, Today, Агентс...
                              ...                        
8258    [Как, мы, писали, еще, весной, для, увеличения...
8259    [Но, молодой, министр, национальной, экономики...
8260    [В, ЕНПФ, назначен, новый, председатель, правл...
8261    [В, Алматы, у, отделения, банка, произошло, на...
8262    [НПП, РК, «Атамекен», предлагает, создать, Нац...
Name: text, Length: 8263, dtype: object

In [9]:
pd.DataFrame(data)

,text
0,"[Досудебное, расследование, по, факту, покупки..."
1,"[Медики, рассказали, о, состоянии, пострадавше..."
2,"[Прошел, почти, год, как, железнодорожным, опе..."
3,"[По, итогам, 12, месяцев, 2016, года, на, терр..."
4,"[Астана, 21, ноября, Kazakhstan, Today, Агентс..."
...,...
8258,"[Как, мы, писали, еще, весной, для, увеличения..."
8259,"[Но, молодой, министр, национальной, экономики..."
8260,"[В, ЕНПФ, назначен, новый, председатель, правл..."
8261,"[В, Алматы, у, отделения, банка, произошло, на..."


In [10]:
data_train.to_csv('data_train_clean_text.csv', index = False)

In [11]:
data.to_csv('data_train_message_text_process.csv', index = False)

In [4]:
data1 = pd.read_csv('data_train_clean_text.csv')

In [5]:
def text_process(mess):
#     # Check characters to see if there are punctuations 
#     no_punctuation=[char for char in mess if char not in string.punctuation]
#     # now form the sentence
    no_punctuation=''.join(mess)
    # Now eliminate any stopwords
    return[word.lower() for word in no_punctuation.split()]
#     return no_punctuation

In [6]:
data = data1['clean_text'].apply(text_process)
data

0       [досудебн, расследован, факт, покупк, енпф, па...
1       [медик, рассказа, состоян, пострада, мужчин, к...
2       [пройт, год, железнодорожн, оператор, запрет, ...
3       [итог, месяц, год, территор, республик, выпуст...
4       [аста, ноябр, kazakhstan, today, агентств, рк,...
                              ...                        
8258    [писа, весн, увеличен, сбыт, такж, избеган, пе...
8259    [молод, министр, национальн, экономик, бишимба...
8260    [енпф, назнач, нов, председател, правлен, един...
8261    [алмат, отделен, банк, произойт, нападен, стре...
8262    [нпп, рк, атамек, предлага, созда, национальн,...
Name: clean_text, Length: 8263, dtype: object

In [26]:
# !pip3 install gensim

In [12]:
from gensim.models import Word2Vec
w2v_model = Word2Vec(
    min_count=10,
    window=2,
    vector_size=300,
    negative=10,
    alpha=0.03,
    min_alpha=0.0007,
    sample=6e-5,
    sg=1)

In [13]:
w2v_model.build_vocab(data)

In [15]:
w2v_model.train(data, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

(50092135, 90434190)

In [19]:
# w2v_model.init_sims(replace=True)

In [17]:
w2v_model.wv.most_similar(positive=["х"])

[('хх', 0.331299364566803),
 ('переплат', 0.3288971781730652),
 ('токсиколог', 0.32131096720695496),
 ('xx', 0.3181544840335846),
 ('барачн', 0.3111740052700043),
 ('kulanshi', 0.30679190158843994),
 ('газовоздушн', 0.3040841519832611),
 ('пятнадца', 0.303924560546875),
 ('приблизительн', 0.2980707287788391),
 ('резервуарн', 0.29701241850852966)]

In [18]:
w2v_model.wv.vectors.shape

(14416, 300)

In [19]:
w2v_model.wv.vectors

array([[ 0.01300017,  0.12605561,  0.01273356, ...,  0.01182407,
         0.1165512 , -0.09134565],
       [ 0.00589335,  0.2540365 , -0.05305502, ..., -0.13445063,
        -0.21756735, -0.14814591],
       [-0.02949037,  0.04986649, -0.04000168, ..., -0.03994336,
         0.01483119, -0.00037922],
       ...,
       [ 0.04874252, -0.23005418,  0.00988154, ..., -0.00546055,
        -0.37245917,  0.10539123],
       [ 0.14578563,  0.37379646,  0.33995473, ..., -0.18594214,
         0.18681559,  0.14370292],
       [-0.05281521,  0.34247145,  0.05633693, ...,  0.130997  ,
        -0.10204629,  0.00074941]], dtype=float32)

In [34]:
# save model in ASCII (word2vec) format
filename1 = 'w2v_model.txt'
w2v_model.wv.save_word2vec_format(filename1, binary=False)

In [35]:
import os

embeddings_i = {}
f = open(os.path.join('', 'w2v_model.txt'),  encoding = "utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:])
    embeddings_i[word] = coefs
f.close()

In [37]:
# embeddings_i

In [38]:
from tqdm.auto import tqdm
docs_vectors2 = pd.DataFrame() # creating empty final dataframe
# stopwords = nltk.corpus.stopwords.words('english') # removing stop words
for doc in tqdm(data):#.str.lower().str.replace('[^a-z ]', ''): # looping through each document and cleaning it
    temp2 = pd.DataFrame()  # creating a temporary dataframe(store value for 1st doc & for 2nd doc remove the details of 1st & proced through 2nd and so on..)
    for word in doc: # looping through each word of a single document and spliting through space
#         if word not in STOPWORDS: # if word is not present in stopwords then (try)
        try:
                    word_vec2 = embeddings_i[word] # if word is present in embeddings(goole provides weights associate with words(300)) then proceed
                    temp2 = temp1.append(pd.Series(word_vec2), ignore_index = True) # if word is present then append it to temporary dataframe
        except:
                    pass
    doc_vector2 = temp2.mean() # take the average of each column(w0, w1, w2,........w300)
    docs_vectors2 = docs_vectors2.append(doc_vector2, ignore_index = True) # append each document value to the final dataframe
docs_vectors2.shape

  0%|          | 0/8263 [00:00<?, ?it/s]

(8263, 0)

In [32]:
# w2v_model.wv.index_to_key

In [21]:
# data.values.tolist()

In [52]:
# import string
# from nltk.tokenize import word_tokenize
# from nltk.corpus import stopwords
# import nltk
# nltk.download('punkt')

# review_lines = list()
# lines = data.values.tolist()

# for line in lines:   
#     tokens = word_tokenize(line)
#     # convert to lower case
#     tokens = [w.lower() for w in tokens]
#     # remove punctuation from each word    
#     table = str.maketrans('', '', string.punctuation)
#     stripped = [w.translate(table) for w in tokens]
#     # remove remaining tokens that are not alphabetic
#     words = [word for word in stripped if word.isalpha()]
#     # filter out stop words    
#     stop_words = set(stopwords.words('russian'))
#     words = [w for w in words if not w in stop_words]
#     review_lines.append(words)

In [18]:
# len(review_lines)

8263

In [22]:
# review_lines

In [8]:
import gensim 

EMBEDDING_DIM = 300
# train word2vec model
model = gensim.models.Word2Vec(sentences=data.values.tolist(), vector_size=EMBEDDING_DIM, window=1, workers=4, min_count=1)

In [ ]:
model.wv.vectors.shape

In [41]:
len(model.wv.index_to_key)

62461

In [42]:
# save model in ASCII (word2vec) format
filename = 'russian_embedding_word2vec.txt'
model.wv.save_word2vec_format(filename, binary=False)

In [43]:
import os

embeddings_index = {}
f = open(os.path.join('', 'russian_embedding_word2vec.txt'),  encoding = "utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:])
    embeddings_index[word] = coefs
f.close()

In [25]:
# total_reviews = data_train['text'].values
# max_length = max([len(s.split()) for s in total_reviews])

In [26]:
# print(max_length)

48146


In [29]:
# embeddings_index

In [ ]:
from tqdm.auto import tqdm
docs_vectors1 = pd.DataFrame() # creating empty final dataframe
# stopwords = nltk.corpus.stopwords.words('english') # removing stop words
for doc in tqdm(data):#.str.lower().str.replace('[^a-z ]', ''): # looping through each document and cleaning it
    temp1 = pd.DataFrame()  # creating a temporary dataframe(store value for 1st doc & for 2nd doc remove the details of 1st & proced through 2nd and so on..)
    for word in doc: # looping through each word of a single document and spliting through space
#         if word not in STOPWORDS: # if word is not present in stopwords then (try)
        try:
                    word_vec1 = embeddings_index[word] # if word is present in embeddings(goole provides weights associate with words(300)) then proceed
                    temp1 = temp1.append(pd.Series(word_vec1), ignore_index = True) # if word is present then append it to temporary dataframe
        except:
                    pass
    doc_vector1 = temp1.mean() # take the average of each column(w0, w1, w2,........w300)
    docs_vectors1 = docs_vectors1.append(doc_vector1, ignore_index = True) # append each document value to the final dataframe
docs_vectors1.shape

  0%|          | 0/8263 [00:00<?, ?it/s]

In [7]:
import gensim
url = "model.bin"
embeddings = gensim.models.KeyedVectors.load_word2vec_format(url, binary=True)

In [8]:
xxx = ' '.join(embeddings.index_to_key)

In [11]:
from tqdm.auto import tqdm
import re
docs_vectors = pd.DataFrame() # creating empty final dataframe
# stopwords = nltk.corpus.stopwords.words('english') # removing stop words
for doc in tqdm(data):#.str.lower().str.replace('[^a-z ]', ''): # looping through each document and cleaning it
    temp = pd.DataFrame()  # creating a temporary dataframe(store value for 1st doc & for 2nd doc remove the details of 1st & proced through 2nd and so on..)
    for word in doc: # looping through each word of a single document and spliting through space
#         if word not in STOPWORDS: # if word is not present in stopwords then (try)
            try:
                        e = r'{}\w+'.format(word)
                        prog = re.compile(e)
                        r = re.search(prog, xxx)
                        t = r.group(0)
           
                        word_vec = embeddings[t] # if word is present in embeddings(goole provides weights associate with words(300)) then proceed
                        temp = temp.append(pd.Series(word_vec), ignore_index = True) # if word is present then append it to temporary dataframe
            except:
                        pass
    doc_vector = temp.mean() # take the average of each column(w0, w1, w2,........w300)
    docs_vectors = docs_vectors.append(doc_vector, ignore_index = True) # append each document value to the final dataframe
docs_vectors.shape

  0%|          | 0/8263 [00:00<?, ?it/s]

(8263, 300)

In [12]:
docs_vectors

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.031685,0.178949,0.009137,-0.135201,-0.060891,0.134090,-0.069089,-0.120070,0.032332,-0.242677,...,0.068924,-0.045049,0.077332,0.143879,-0.093730,-0.151954,-0.086196,0.119717,0.186446,0.122009
1,0.000640,0.171886,-0.067648,-0.059812,-0.138297,0.147603,0.000309,-0.218301,0.074575,-0.205155,...,0.066938,-0.005971,0.080198,0.072171,0.038017,-0.127977,-0.036574,0.136970,0.072631,0.127916
2,0.002386,0.005910,-0.029268,-0.117138,-0.076510,0.156052,0.016359,-0.190923,0.097740,-0.199994,...,0.042932,-0.122284,0.049808,0.223850,-0.132330,-0.134603,0.073164,0.064367,0.022614,0.066349
3,0.052623,0.024831,-0.014060,-0.137594,-0.051393,0.056387,-0.019196,-0.137196,0.145147,-0.241803,...,0.089164,-0.088749,0.049736,0.263677,-0.156651,-0.016040,-0.120103,0.169930,0.086296,0.087752
4,0.022835,0.096034,-0.044885,-0.027258,-0.043773,0.082485,-0.013675,-0.225652,0.117113,-0.168474,...,0.048638,-0.102031,0.092301,0.159423,-0.072504,-0.125476,-0.024196,0.136839,0.169404,0.022702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8258,-0.009181,0.005057,-0.042632,-0.082400,-0.086237,0.080103,0.031975,-0.170380,0.189781,-0.230101,...,0.164937,-0.141198,-0.009332,0.225098,-0.105520,-0.087568,-0.068922,0.134096,0.065158,0.041643
8259,0.048767,0.059226,-0.008510,-0.077230,-0.068700,0.131523,0.066039,-0.185712,0.076746,-0.197847,...,0.122072,-0.081235,0.005561,0.206319,-0.109283,-0.063879,-0.032915,0.148445,0.043832,0.032009
8260,-0.030871,0.210113,0.004565,-0.050671,0.038129,0.044490,-0.133662,-0.182833,-0.081432,-0.128037,...,0.102866,0.100666,-0.084105,0.060187,-0.109676,0.032487,-0.083420,0.083041,0.218849,0.003763
8261,0.021308,0.170105,-0.091837,-0.090477,-0.070167,0.187116,0.019334,-0.259912,0.014777,-0.184171,...,0.046713,-0.109891,0.076076,0.100614,0.017036,-0.092438,-0.021866,0.180605,0.055763,0.139543


In [13]:
docs_vectors['Sentiment'] = data_train['sentiment']
docs_vectors = docs_vectors.dropna()

In [14]:
docs_vectors['Sentiment'] = docs_vectors['Sentiment'].map(lambda x: 0 if x == 'negative' else 1 if x == 'positive' else 2)

In [15]:
pd.isnull(docs_vectors).sum().sum()

0

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

train_x, test_x, train_y, test_y = train_test_split(docs_vectors.drop('Sentiment', axis = 1),
                                                   docs_vectors['Sentiment'],
                                                   test_size = 0.2,
                                                   random_state = 1)
train_x.shape, train_y.shape, test_x.shape, test_y.shape

((6610, 300), (6610,), (1653, 300), (1653,))

In [17]:
model1 = AdaBoostClassifier(n_estimators=800, random_state = 1)
model1.fit(train_x, train_y)
test_pred1 = model1.predict(test_x)
accuracy_score(test_y, test_pred1)

0.6110102843315185

In [18]:
import pickle

with open("model_w2v_AdaBoostClassifier.pkl", "wb") as f:
    pickle.dump(model1, f)